#### <div style="text-align: right">PHY-3500</div>
# **<center>PROJET | MÉTHODE PSEUDO-SEPCTRALE : PROPAGATION D'UN PAQUET D'ONDE</center>**
## <center>Bastien Bédard, Camille Poitras et Jean-Simon Côté</center>
### <center>*10 mai 2023*</center>
---

In [1]:
import cv2
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from numpy.fft import fft, ifft, fft2, ifft2
import os
import pandas as pd
import pathlib
from PIL import Image
import scipy.constants as cte
from scipy.special import factorial as facto
import time
import imageio
from moviepy.editor import ImageSequenceClip
import matplotlib.colors as mcolors

plt.rcParams.update({'font.size': 16})
pd.set_option("display.precision", 16)

# Définir le rayon de Bohr
a0 = cte.physical_constants["Bohr radius"][0]

In [2]:
# Créer une cmap de transparence
cmap_trans = mcolors.LinearSegmentedColormap.from_list("", ["#FF000000",
                                                            'black'])
# Créer une cmap personnalisée en combinant deux cmap existante
colors1 = plt.cm.hot_r(np.linspace(0.82, 1, 40))
colors2 = plt.cm.jet(np.linspace(0, 1, 158))

# Combine les deux cmap pour en faire une nouvelle
colors = np.vstack((colors2, colors1))
mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)

cblue = '#14385A'
cyellow = '#FEB703'
corange = '#FB8500'

## 1. Unidimensionnel
La condition initiale d'un paquet d'onde en une dimension $\psi(x, 0)$ peut être décrite par la relation suivante:
\begin{equation}
    \psi(x,0) = \left[\frac{1}{2\pi(\Delta x)^2}\right]^{1/4}\text{exp}\left({ik_0x - \frac{(x-x_0)^2}{4(\Delta x)^2}}\right)
\end{equation}

\begin{equation}
\psi(x,t) \approx e^{-i\mathbf{\hat{V}}\Delta t/2\hbar}\mathcal{F}^{-1}\left[e^{-iT(k)\Delta t/\hbar}\mathcal{F}\left[e^{-i\mathbf{\hat{V}}\Delta t/2\hbar}\psi(x,t_0)\right]\right]
\end{equation}

Afin d'implémenter numériquement la méthode pseudo-spectral, différentes classes et fonctions ont été implémentées. D'abord, la classe `Param` permet d'initialiser un object lié aux conditions initiales du système simulé. C'est-à-dire la dimension du système `xmax`, le nombre de points pour les calculs `nb`, le pas de temps entre chaque calcul d'évolution temporelle `dt` ainsi que la précision désiré sur ce pas de temps `precision`. De plus, cette classe initialise quelques paramètres de la gaussienne initiale soit sa position `x0`, sa largeur `largeur` et son nombre d'onde `k0`. 

In [3]:
class Param:
    """ Classe d'initialisation des conditions initiales """
    def __init__(self, xmax=a0, nb=500, dt=0.05, precision=500, x0=0,
                 largeur=a0/30, k0=0):
        self.xmax = xmax
        self.nb = nb
        self.dt = dt
        self.precision = precision
        self.x0 = x0
        self.largeur = largeur
        self.k0 = k0
        self.x = np.arange(-xmax+xmax/nb, xmax, 2*xmax/nb)
        self.k = np.abs(np.concatenate((np.arange(0, np.ceil(nb/2),
                    dtype=complex), np.arange(-np.floor(nb/2), 0, 
                                    dtype=complex))) * np.pi / xmax)

Par la suite, puisqu'il était prévu de simuler plusieurs potentiels différents, une fonction `potentiel_param` a été créée afin d'implémenter facillement les paramètres des différents potentiels étudiés. Cette fonction utilise donc un dictionnaire pour associer la forme de 6 types de potentiel à un nom. Donc, en fonction du nom de potentiel donné à la fonction, un `numpy array` est créé avec les bons paramètres pour celui-ci. Deux de ces paramètres sont les ordres de grandeur du pas de temps et du potentiel. Ceux-ci sont calculés de façon à permettre un affichage pertinent lors de la génération de graphique. Un des arguments optionnels que la fonction prend en entrée est `boite` qui est un booléen qui, si posé à True, permet d'ajouter au potentiel choisi un potentiel très élevé à quelques des premiers et derniers éléments de façon à s'assurer que la particule soit confinnée dans la région d'intéret si désiré.

In [4]:
def potentiel_param(par=Param(), nom_pot='potentiel nul', prop=0, boite=False,
                    hmax=1, hmin=0):
    """ Calcule les ordres de grandeur voulu 
                            pour le pas de temps et le potentiel """
    # Calcul de l'ordre de grandeur adéquat du pas de temps
    ordre_t = cte.hbar * np.max(par.k**2) / (0.2 * cte.m_e * par.nb)
    
    # Calcul de l'ordre de grandeur adéquat du potentiel
    ordre_v = cte.hbar*ordre_t*hmax
    
    # Initialisation de différents potentiels
    pot_nul, pot_const = np.zeros(par.nb), np.ones(par.nb)
    pot_quad = par.x**2*3e20/ordre_v
    pot_marche = np.round(np.linspace(0, 0.8, par.nb))
    pot_carre = np.round(np.abs(np.linspace(-1, 1, par.nb)))
    pot_bar = np.round(np.abs(np.linspace(-1, 1, par.nb))-0.503)*-1

    # Dictionnaire des array pour certains potentiels
    dict_potentiel = {'potentiel nul': pot_nul,
              'potentiel constant': pot_const,
              'potentiel quadratique' : pot_quad,
              'potentiel carre' : pot_carre, 
              'potentiel marche': pot_marche,
              'potentiel barriere' : pot_bar*10}
    
    # Array des paramètres du système en fonction du potentiel choisi
    params = np.array([ordre_t, ordre_v*dict_potentiel[nom_pot],
                    prop * 2e12 * par.nb/5e2, 0, par.precision], dtype=object)

    # Ajoute un potentiel 'infini' sur le contour de façon à former
    #       une boite de potentiel si désiré
    if boite:
        params[1][np.append(np.array(range(int(par.nb*0.05))),
                -1*np.array(range(int(par.nb*0.05)+1))[1:])] = 1e200*ordre_v
    return list(np.append(params, np.array(ordre_v)))

Pour le calcul de la méthode pseudo-spectral en une dimension, une classe `Operateurs` est créée afin d'initialiser un objet avec des opérateurs d'énergie cinétique et d'énergie potentielle sous forme de matrice nulle. La classe initialise aussi une matrice nulle associée à la fonction d'onde.

Ensuite, une fonction `init_1D` permet de prendre un objet de la classe `Operateurs` et de calculer la valeur des deux opérateurs d'énergie et de la fonction d'onde à partir des conditions intiales du système données en entrée par un objet de la classe `Param`.

In [5]:
class Operateurs:
    """ Initialise des opérateurs en une dimension pour l'énergie potentielle, 
         l'énergie cinétique et la fonction d'onde """
    def __init__(self, nb):
        self.Er = np.zeros(nb, dtype=complex)
        self.Ep = np.zeros(nb, dtype=complex)
        self.fc_onde = np.zeros(nb, dtype=complex)

def init_1D(par, potentiel, m=cte.m_e):
    """ Calcul les opérateurs selon les conditions initiales du système pour
                  un potentiel donné et les attribue à un object Operateurs"""
    dt = par.dt/par.precision
    opr = Operateurs(par.nb)
    opr.fc_onde = ((1/(2*np.pi*par.largeur**2)**(1/4))*
                np.exp(1j*par.k0*par.x - (par.x-par.x0)**2/(4*par.largeur**2)))
    opr.Er = np.exp(-1j * potentiel * dt / (2 * cte.hbar))
    opr.Ep = np.exp(-1j * cte.hbar * par.k**2 * dt / (2*m))
    return opr

Une fois que la fonction d'onde initiale est calculée, ainsi que les deux opérateurs d'énergie, la fonction `separation_operateurs` applique la méthode pseudo-spectrale pour calculer la propagation temporelle pour un pas de temps avec un précision `par.precision`.

In [6]:
def separation_operateurs(par, opr):
    """ Calcule la fonction d'onde par la méthode pseudo-spectral à répétition 
                                          selon la précision donnée """
    for _ in range(par.precision):
        opr.fc_onde = ifft(fft(opr.fc_onde * opr.Er) * opr.Ep) * opr.Er
    return opr.fc_onde

Maintenant que toutes les fonctions et classes nécessaires pour l'implémentation numérique de la méthode pseudo-spectrale sont définies, il est possible de tester la méthode avec des exemples. 

Pour ce faire, une fonction d'affichage de graphique `graphique_1D` et une fonction de sauvegarde de ces graphiques `graphique_1Dsave` ont été créées. La première fonction prend en entrée la fonction d'onde et le potentiel puis affiche la partie réelle, imaginaire et la densité de probabilité de la fonction d'onde dans un graphique. En plus, celle-ci représente la forme du potentiel. La seconde fonction appelle la première pour faire la même chose mais à la place d'afficher les figures elle les enregistre en png.

In [7]:
def graphique_1D(par, solution, potentiel, iter, norm=1):
    """ Affiche le graphique de la fonction d'onde et 
                                de la densité de probabilité en 1D """
    # Graphique de la densité de probabilité
    plt.plot(par.x, (np.sqrt(np.conj(solution)*solution).real)/norm, c=cblue,
          linewidth=2, zorder=10, label='Densité')
    # Graphique de la partie réelle de la fonction d'onde
    plt.plot(par.x, np.real(solution)/norm, c='red', linewidth=1,
             label='Réelle')
    # Graphique de la partie imaginaire de la fonction d'onde
    plt.plot(par.x, np.imag(solution)/norm, c='limegreen', linewidth=1,
             label='Imaginaire', zorder=5)
    
    # Affichage de la forme du potentiel par dessus les autres fonctions
    potentiel[potentiel!=0] = 1
    plt.plot(par.x, potentiel, c='black')

    # Paramètres d'affichage
    plt.xlabel('$%s$' %r'x [m]')
    plt.ylabel('Densité normalisée ' + '$%s$' %r'\sqrt{\psi(x, t)*\psi(x, t)}')
    plt.xlim(-par.xmax, par.xmax)
    plt.ylim(-1, 1)
    plt.xticks([-a0, -a0/2, 0, a0/2, a0], ['$%s$' %r'-a_0',
                      '$%s$' %r'\frac{-a_0}{2}', 0,
                       '$%s$' %r'\frac{a_0}{2}', '$%s$' %r'a_0'])
    plt.legend(loc='lower left', fontsize=13)
    plt.tick_params(direction='in')
    plt.twinx()
    plt.yticks([-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1])
    plt.tick_params(direction='in')
    plt.ylabel("Amplitude normalisée " + '$%s$' %r'\psi(x, t)')
    plt.annotate('t = {:g} s'.format(float('{:.4g}'.format(par.dt*iter))),
                                                            (2.8e-11, 0.9))

In [8]:
def graphique_1Dsave(par, solution, potentiel, iter, norm):
    """ Génère et sauvegarde les images pour les vidéos """
    plt.figure(figsize=(12, 6), dpi=120)
    plt.xlim(-0.4, 0.4)
    graphique_1D(par, solution, potentiel, iter, norm)
    plt.savefig(f"figure{iter}.png")
    plt.clf()

Avec les fonctions qui permettent d'afficher les graphiques, il est possible de faire les calculs de la méthode pseudo-spectrale pour un contexte quelconque avec la fonction `graph_test` qui prend en argument les conditions initiales sous forme d'un objet de la classe `Param`, le potentiel et le nombre de pas de temps désiré.

In [9]:
def graph_test(par, step, potentiel):
    """ Calcule et affiche la fonction d'onde pour calibrer 
                    les paramètres avant de générer une vidéo """
    plt.figure(figsize=(10, 6), dpi=120)

    # initialise les objets pour les différents opérateurs 
    #                                       aux conditions initiales
    opr = init_1D(par, potentiel)
    # Facteur de normalisation pour la densité de probabilité
    norme = np.max(np.sqrt((np.conj(opr.fc_onde)*opr.fc_onde).real))

    # Calcule et affiche la fonction d'onde pour le nombre de pas demandé
    graphique_1D(par, opr.fc_onde, potentiel, 0, 2*norme)
    for i in range(1, step):
          solution = separation_operateurs(par, opr)
          if i>step-2:
              graphique_1D(par, solution, potentiel, i, 2*norme)
    plt.show()

En affichant le graphique de la propagation avec la fonction `graph_test` il est possible d'ajuster les différents paramètres pour obtenir des conditions initiales qui vont permettre de bien représenter la propagation temporelle dans une vidéo. Celle-ci peut être faite par la fonction `video_simulation` qui fonctionne de la même façon que `graph_test`, mais qui enregistre les graphiques et en fait une vidéo à la place de les afficher.


In [10]:
def video_simulation(par, potentiel, frame=50, fps=5, file_name='Simulation'):
    """ Génère un vidéo de l'évolution de la fonction d'onde et de
                                 la densité de probabilité dans le temps """
    # initialise les objets pour les différents opérateurs 
    #                                       aux conditions initiales
    opr = init_1D(par, potentiel)

    # Facteur de normalisation pour la densité de probabilité
    norm = np.max((np.conj(opr.fc_onde)*opr.fc_onde).real**0.5)

    # Calcule et enregistre le graphique de chaque pas de temps
    graphique_1Dsave(par, opr.fc_onde, potentiel, 0, 2*norm)
    for i in range(1, frame):
        solution = separation_operateurs(par, opr)
        graphique_1Dsave(par, solution, potentiel, i, 2*norm)
    
    # Convertir les images en une vidéo
    image_files = [f"figure{i}.png" for i in range(frame)]
    images = [imageio.v2.imread(file) for file in image_files]
    clip = ImageSequenceClip(images, fps=fps)
    clip.write_videofile(f"{file_name}.mp4")
    # Supprimer les images dans le dossier pour conserver que la vidéo
    for i in range(frame):
        os.remove(f"figure{i}.png")

Un exemple de test de graphique pour un potentiel carré en 1D est visible ici si le False est remplacé par True.

In [11]:
# Graphique de calibration d'unu potentiel carré en 1D
if False:
    par = Param(nb=500, precision=50)
    dt, potentiel, par.k0, par.x0, par.precision, ordre_v = potentiel_param(par, 
                                        'potentiel carre', prop=5, boite=False)
    par.dt=1/dt
    graph_test(par, 1, potentiel*1e20)

Un exemple création de vidéo est visible ici si le False est remplacé par True.

In [12]:
# Génère une vidéo pour un potentiel carré en 1D
if False:
    par = Param(nb=500, precision=50)
    dt, potentiel, par.k0, par.x0, par.precision, ordre_v= potentiel_param(par,
                                        'potentiel carre', prop=3, boite=False)
    par.dt = 0.1/dt
    video_simulation(par, potentiel, frame = 60, fps=40,
                          file_name='Simulation_Potentiel_nul(k0)')

## 2. Bidimensionnel
La condition initiale du paquet d'onde en deux dimensions $\psi(x, y, 0)$ peut être décrite par la relation suivante:
\begin{equation}
    \psi(x,y,0) = \left[\frac{1}{2\pi(\Delta x)^2(\Delta y)^2}\right]^{1/4}\text{exp}\left({ik_{x0}x + ik_{y0}y - \frac{(x-x_0)^2}{4(\Delta x)^2}- \frac{(y-y_0)^2}{4(\Delta y)^2}}\right)
\end{equation}

\begin{equation}
\psi(x,y,t) \approx e^{-i\mathbf{\hat{V}}\Delta t/2\hbar}\mathcal{F}^{-1}\left[e^{-iT(k)\Delta t/\hbar}\mathcal{F}\left[e^{-i\mathbf{\hat{V}}\Delta t/2\hbar}\psi(x, y,t_0)\right]\right]
\end{equation}

Pour le calcul de la méthode pseudo-spectrale en 2D, il est possible de former des potentiels symétriques en deux dimensions à partir des potentiels définis plus tôt dans la fonction `potentiel_param` en 1D. Il est aussi possible d'utiliser d'autres formes de potentiel en 2D qui ne sont pas symétriques. Afin de pouvoir représenter n'importe quel potentiel, la fonction `contour_image` prend en entrée une image et la représente sous forme d'une matrice 2D de 1 et de 0. Ou l'extérieur de l'object dans l'image est rempli de 1 et l'intérieur de 0. Celle-ci détecte essentiellement le plus grand contour de l'image fournie.

In [13]:
def contour_image(direction_du_fichier, puissance_gaussienne=5):
    #Télécharge l'image en array, le 0 indique qu'elle est en grayscale
    image_grise_originale = cv2.imread(direction_du_fichier, 0) 

    #Convolution avec des gaussiennes pour faciliter la détection des contours
    image = cv2.GaussianBlur(image_grise_originale.copy(), (5,5), 0) 

    # Applique d'autres convolutions avec des gaussiennes,
    #    chaque convolution réduit le bruit autour du contour,
    #        mais enlève des détails du contour
    for n in range(puissance_gaussienne): 
        image = cv2.GaussianBlur(image, (5,5), 0)
    
    # Détecte en regardant la moyenne des greyscale de la première ligne
    #    si le fond est plutôt blanc ou noir. 
    #      S'il est noir, le grey scale est inversé pour que le fond soit noir
    if image[0].mean()>100:         
        image = cv2.bitwise_not(image)
    
    # Trouve les contours dans l'image
    contours, hierarchie = cv2.findContours(image, cv2.RETR_EXTERNAL,
                                            cv2.CHAIN_APPROX_NONE) 
    
    # Classe les contours de la plus grande à la plus petite aire
    contours = sorted(contours, key=cv2.contourArea)[::-1] 

    # Remplis tous les contours de noir
    cv2.fillPoly(image, contours, color=(0,0,0)) 

    # Remplis uniquement le contour avec la plus grande aire de blanc
    cv2.fillPoly(image, [contours[0]], color=(255, 255, 255)) 

    # Transforme tous les pixels qui ne sont pas purement blanc en noir,
    #       la matrice retournée contient donc uniquement des 0 et des 255
    _, image_binaire = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY) 

    # Divise par 255 la matrice afin de retourner une matrice de 0 et de 1
    #                                          ayant la forme du potentiel
    return cv2.bitwise_not(image_binaire)/255 

Pour le calcul de la méthode pseudo-spectrale en deux dimension, une classe `Operateurs2D` est créée de la même façon qu'en 1D mais initialise des matrices 2D à la place.

Ensuite, une fonction `init_2D` permet de prendre un objet de la classe `Operateurs2D` et de calculer la veleur des opérateurs et de la fonction d'onde en 2D de la même façon que `init_1D`.

In [14]:
class Operateurs2D:
    """ Initialise des opérateurs en deux dimensions pour l'énergie potentielle, 
         l'énergie cinétique et la fonction d'onde """
    def __init__(self, nb_x, nb_y):
        self.Er = np.zeros((nb_y, nb_x), dtype=complex)
        self.Ep = np.zeros((nb_y, nb_x), dtype=complex)
        self.fc_onde = np.zeros((nb_y, nb_x), dtype=complex)

def init_2D(parx, pary, potentiel, m=cte.m_e):
    """ Calcul les opérateurs selon les conditions initiales du système pour
                  un potentiel donné et les attribuent à un object Operateurs"""
    dt = parx.dt/parx.precision
    #dt = 1/max(parx.dt, pary.dt)
    opr2d = Operateurs2D(parx.nb, pary.nb)
    mat_xx, mat_yy = np.meshgrid(parx.x, pary.x)
    opr2d.fc_onde = ((1/(2*np.pi*parx.largeur**2*pary.largeur**2)**(1/4)) 
                    *np.exp(1j*parx.k0*mat_xx+1j*pary.k0*mat_yy - 
                            (mat_xx-parx.x0)**2/(4*parx.largeur**2) - 
                            (mat_yy-pary.x0)**2/(4*pary.largeur**2)))
    k_xx, k_yy = np.meshgrid(parx.k, pary.k)
    k_2D = np.sqrt(k_xx**2+k_yy**2)
    opr2d.Er = np.exp(-1j * potentiel * dt / (2 * cte.hbar))
    opr2d.Ep = np.exp(-1j * cte.hbar * np.power(k_2D, 2) * dt / (2*m))
    return opr2d

Une fois que la fonction d'onde initiale est calculée, ainsi que les deux opérateurs d'énergie, la fonction `separation_operateurs2d` applique la méthode pseudo-spectrale pour calculer la propagation temporelle pour un pas de temps avec une précision `par.precision`.

In [15]:
def separation_operateurs2d(par, opr2d):
    """ Calcule la fonction d'onde par la méthode pseudo-spectrale à répétition 
                                          selon la précision donnée """
    for i in range(par.precision):
      opr2d.fc_onde = ifft2(fft2(opr2d.fc_onde * opr2d.Er)* opr2d.Ep)* opr2d.Er
    return opr2d.fc_onde

Maintenant que toutes les fonctions et classes nécessaires pour l'implémentation numérique de la méthode pseudo-spectrale en 2D sont définies, il est possible de tester la méthode avec des exemples. 

Pour ce faire, une fonction d'affichage de graphique `graphique_2D` et une fonction de sauvegarde de ces graphiques `graphique_2Dsave` ont été créées. Ces fonctions remplissent le même rôle que les fonctions `graphique_1D` et `graphique_1Dsave` en une dimension.

In [16]:

def graphique_2D(par,fig, solution, dens_prob,potentiel, iter, norm):
    """ Affiche le graphique de la fonction d'onde et 
                                     de la densité de probabilité en 2D """
    fig.suptitle("Amplitude normalisée " + '$%s$' %r'\psi(x, y, t)')

    # Graphique de la densité de probabilité
    gn = plt.imshow(dens_prob/norm, vmin=0, alpha = 0.90,
                    vmax=1, cmap=mymap)
    
    # Normalise le potentiel
    potentiel[potentiel!=0] = 1

    # Affichage de la forme du potentiel par dessus la densité
    plt.imshow(potentiel.real, alpha=0.45, cmap=cmap_trans)

    # Paramètres d'affichage
    plt.tick_params(direction='in')
    plt.xlabel('$%s$' %r'x [m]')
    plt.ylabel('$%s$' %r'y [m]')
    # plt.xticks([0, par.nb/4, par.nb/2, 3*par.nb/4, par.nb],
    #                    ['$%s$' %r'-a_0', '$%s$' %r'\frac{-a_0}{2}', 0,
    #                    '$%s$' %r'\frac{a_0}{2}', '$%s$' %r'a_0'])
    # plt.yticks([0, par.nb/4, par.nb/2, 3*par.nb/4, par.nb],
    #                 ['$%s$' %r'-a_0', '$%s$' %r'\frac{-a_0}{2}', 0,
    #                    '$%s$' %r'\frac{a_0}{2}', '$%s$' %r'a_0'])
    # plt.tick_params(pad = 7)
    norm = mcolors.Normalize(vmin=0, vmax=1)
    scalar_map = plt.cm.ScalarMappable(norm=norm, cmap=mymap)
    cax = [0.0, 0.2, 0.4, 0.4, 0.6, 0.8, 1.0]
    fig.colorbar(scalar_map, ticks=cax)#, location='bottom'
    plt.annotate('t = {:g} s'.format(float('{:.4g}'.format(par.dt*iter)))+ '\n',
                 (0.3*par.nb, 0.05*par.nb))#-0.1*par.nb

In [17]:
def graphique_2Dsave(par,solution, dens_prop, potentiel, iter, norm):
    """ Génère et sauvegarde les images pour les vidéo """
    fig = plt.figure(figsize=(10, 6), dpi=150)
    graphique_2D(par,fig, solution, dens_prop, potentiel, iter, norm)
    plt.savefig(f"figure{iter}.png")
    plt.clf()

Avec les fonctions qui permettent d'afficher les graphiques, il est possible de faire les calculs de la méthode pseudo-spectrale pour un contexte quelconque avec la fonction `graph_test2D` sur le même principe qu'en une dimension.

In [18]:
def graph_test_2D(parx, pary, potentiel, potentiely, step=1):
    """ Calcule et affiche la densité de probabilité pour calibrer 
                    les paramètres avant de générer une vidéo """
    fig = plt.figure(figsize=(10, 6), dpi=150)
    
    # Détermine si le calcul peut être fait par une combinaison de calcul
    #  en 1D pour x et y (beaucoup plus rapide) ou si le calcul doit être fait 
    #                                  en 2D du à un potentiel asymétrique
    if type(potentiely) is int:
        # Méthode matricielle en 2D
        opr = init_2D(parx, pary, potentiel)
        solutionxy = opr.fc_onde
    else:
        # Méthode de combinaision de 1D en cas de symétrie
        oprx, opry = init_1D(parx, potentiel), init_1D(pary, potentiely)
        repx, repy = np.meshgrid(oprx.fc_onde, opry.fc_onde)
        solutionxy = np.sqrt(repx*repy)
        potx, poty = np.meshgrid(potentiel, potentiely)
        potentiel = (potx+poty)
    
    # Calcule la densité de probabilité
    densite_prob = np.sqrt((np.conj(solutionxy)*solutionxy).real)

    # Facteur de normalisation
    norm = np.max(densite_prob)

    # Affichage du graphique initial si le pas est de 1
    if step == 1: 
        graphique_2D(parx, fig, solutionxy, densite_prob, potentiel, 0, norm)
    
    # Calcule et affichage des graphiques pour le pas demandé
    for i in range(1, step):
        # Détermine si le calcul peut être fait par un combinaison de calcul
        #  en 1D pour x et y (beaucoup plus rapide) ou si le calcul doit 
        #                      être fait en 2D du à un potentiel asymétrique
        if type(potentiely) is int:
            # Méthode matricielle en 2D
            solutionxy = separation_operateurs2d(parx, opr)
        else:
            # Méthode de combinaision de 1D en cas de symétrie
            solutionx = separation_operateurs(parx, oprx)
            solutiony = separation_operateurs(pary, opry)
            repx, repy = np.meshgrid(solutionx, solutiony)
            solutionxy = np.sqrt(repx*repy)
        
        # Calcule la densité de probabilité
        densite_prob = np.sqrt((np.conj(solutionxy)*solutionxy).real)

        # Affiche le dernier graphique calculé pour ajuster les conditions
        #                                                       initiales
        if i>step-2:
            graphique_2D(parx,fig, solutionxy, densite_prob, potentiel, 0, norm)
    plt.show()

En affichant le graphique de la propagation avec la fonction `graph_test2D`, il est possible d'ajuster les différents paramètres pour obtenir des conditions initiales qui vont permettre de bien représenter la propagation temporelle dans une vidéo. Celle-ci peut être faite par la fonction `video_simulation2D` qui fonctionne de la même façon que `graph_test2D`, mais qui enregistre les graphiques et en fait un vidéo à la place de les afficher.


In [19]:
def video_simulation2D(parx, pary, potentiel, potentiely,
                               allframe=1, fps=60, file_name='Simulation'):
    """ Génère un vidéo de l'évolution de la densité de probabilité
                                                          dans le temps """
    
    # Détermine si le calcul peut être fait par une combinaison de calcul
    #  en 1D pour x et y (beaucoup plus rapide) ou si le calcul doit être fait 
    #                                  en 2D du à un potentiel asymétrique
    if type(potentiely) is int:
        # Méthode matricielle en 2D
        opr = init_2D(parx, pary, potentiel)
        solutionxy = opr.fc_onde
    else :
        # Méthode de combinaision de 1D en cas de symétrie
        oprx, opry = init_1D(parx, potentiel), init_1D(pary, potentiely)
        repx, repy = np.meshgrid(oprx.fc_onde, opry.fc_onde)
        solutionxy = np.sqrt(repx*repy)
        potx, poty = np.meshgrid(potentiel, potentiely)
        potentiel = (potx+poty)
    
    # Calcule la densité de probabilité
    densite_prob = np.sqrt((np.conj(solutionxy)*solutionxy).real)

    # Facteur de normalisation
    norm = np.max(densite_prob)

    # Calcule et enregistre les graphiques pour chaque pas demandé
    graphique_2Dsave(parx, solutionxy, densite_prob, potentiel, 0, norm)
    for frame in range(1, allframe):
        # Détermine si le calcul peut être fait par une combinaison de calcul
        #      en 1D pour x et y (beaucoup plus rapide) ou si le calcul doit
        #                       être fait en 2D du à un potentiel asymétrique
        if type(potentiely) is int:
            # Méthode matricielle en 2D
            solutionxy = separation_operateurs2d(parx, opr)
        else:
            # Méthode de combinaision de 1D en cas de symétrie
            solutionx = separation_operateurs(parx, oprx)
            solutiony = separation_operateurs(pary, opry)
            repx, repy = np.meshgrid(solutionx, solutiony)
            solutionxy = np.sqrt(repx*repy)
        
        # Calcul la densité de probabilité
        densite_prob = np.sqrt((np.conj(solutionxy)*solutionxy).real)
        # Génère et enregistre le graphique de la densité de probabilité
        graphique_2Dsave(parx, solutionxy, densite_prob, potentiel, frame, norm)

    # Convertir les images en une vidéo
    image_files = [f"figure{frame}.png" for frame in range(allframe)]
    images = [imageio.v2.imread(file) for file in image_files]
    clip = ImageSequenceClip(images, fps=fps)
    clip.write_videofile(f"{file_name}.mp4")

    # Supprimer les images dans le dossier pour conserver que la vidéo
    for frame in range(allframe):
        os.remove(f"figure{frame}.png")

Quatres exemples de test de graphique pour différents potentiels en 2D sont visibles ici si les False sont remplacés par des True (un à la fois).

In [20]:
# Graphique de calibration pour un potentiel carré avec propagation
if False: 
    parx = Param(precision = 200, nb=1400)
    pary = Param(precision = 200, nb=1400)

    par_potx = potentiel_param(parx, 'potentiel barriere', prop=6, boite=True)
    par_poty = potentiel_param(pary, 'potentiel nul', prop=5, boite=True)
    dtx, potentielx, parx.k0, parx.x0, parx.precision, ordre_v = par_potx
    dty, potentiely, pary.k0, pary.x0, pary.precision, ordre_v = par_poty
    parx.dt=1.3/np.max([dtx, dty])
    pary.dt=1.3/np.max([dtx, dty])
    parx.largeur *= 1.8
    pary.largeur *= 1.8
    parx.x0 -= parx.xmax/13
    pary.x0 -= pary.xmax/2.5
    graph_test_2D(parx, pary, potentielx*300, potentiely, step = 20)

# Graphique de calibration pour un potentiel circulaire sans propagation
if False: 
    parx = Param(precision = 30, nb=600)
    pary = Param(precision = 30, nb=600)

    par_potx = potentiel_param(parx, 'potentiel nul', prop=0, boite=False)
    par_poty = potentiel_param(pary, 'potentiel nul', prop=0, boite=False)
    dtx, potentielx, parx.k0, parx.x0, parx.precision, ordre_v = par_potx
    dty, potentiely, pary.k0, pary.x0, pary.precision, ordre_v = par_poty
    parx.dt=6/np.max([dtx, dty])
    parx.largeur *= 2.5
    pary.largeur *= 2.5
    list_nbx = np.linspace(-1, 1, parx.nb)
    list_nby = np.linspace(-1, 1, pary.nb)
    mat_nbx, mat_nby = np.meshgrid(list_nbx, list_nby)
    potentiel = np.round(np.sqrt(mat_nbx**2+mat_nby**2)-0.45)*ordre_v*1e100
    potentiely = 0
    graph_test_2D(parx, pary, potentiel, potentiely, step = 1)

# Graphique de calibration pour un potentiel circulaire avec propagation
if False: 
    parx = Param(precision = 35, nb=1000)
    pary = Param(precision = 35, nb=1000)

    par_potx = potentiel_param(parx, 'potentiel nul', prop=0, boite=False)
    par_poty = potentiel_param(pary, 'potentiel nul', prop=5, boite=False)
    dtx, potentielx, parx.k0, parx.x0, parx.precision, ordre_v = par_potx
    dty, potentiely, pary.k0, pary.x0, pary.precision, ordre_v = par_poty
    parx.dt=0.4/np.max([dtx, dty])
    parx.largeur *= 1.5
    pary.largeur *= 1.5
    list_nbx = np.linspace(-1, 1, parx.nb)
    list_nby = np.linspace(-1, 1, pary.nb)
    mat_nbx, mat_nby = np.meshgrid(list_nbx, list_nby)
    potentiel = np.round(np.sqrt(mat_nbx**2+mat_nby**2)-0.45)*ordre_v*1e300
    potentiely = 0
    graph_test_2D(parx, pary, potentiel, potentiely, step = 150)

# Graphique de calibration pour un potentiel quelconque 
#                                           calculé à partir d'une image
if False:
    mat = contour_image("Lobster (2).png", puissance_gaussienne=1)
    size = np.shape(mat)
    parx = Param(precision = 35, nb=size[1])
    pary = Param(precision = 35, nb=size[0])

    par_potx = potentiel_param(parx, 'potentiel nul', prop=-1, boite=False)
    par_poty = potentiel_param(pary, 'potentiel nul', prop=0, boite=False)
    dtx, potentielx, parx.k0, parx.x0, parx.precision, ordre_v = par_potx
    dty, potentiely, pary.k0, pary.x0, pary.precision, ordre_v = par_poty
    parx.dt=0.4/np.max([dtx, dty])
    parx.largeur = parx.xmax/30
    pary.largeur = pary.xmax/30
    parx.x0 -= parx.xmax/8
    pary.x0 += pary.xmax/12
    potentiel = mat*ordre_v*1e300
    potentiely = 0
    graph_test_2D(parx, pary, potentiel, potentiely, step = 1)

Un exemple de création d'une vidéo pour un potentiel circulaire 2D est visible ici si le False est remplacé par True.

In [21]:
# Génère une vidéo pour un potentiel circulaire avec propagation en 2D
if False:
    parx = Param(precision = 35, nb=1000)
    pary = Param(precision = 35, nb=1000)
    par_potx = potentiel_param(parx, 'potentiel nul', prop=0, boite=False)
    par_poty = potentiel_param(pary, 'potentiel nul', prop=5, boite=False)
    dtx, potentielx, parx.k0, parx.x0, parx.precision, ordre_v = par_potx
    dty, potentiely, pary.k0, pary.x0, pary.precision, ordre_v = par_poty
    parx.dt=0.4/np.max([dtx, dty])#100 4e-2 200 6e-2 300 1e-1
    parx.largeur *= 1.5
    pary.largeur *= 1.5
    list_nbx = np.linspace(-1, 1, parx.nb)
    list_nby = np.linspace(-1, 1, pary.nb)
    mat_nbx, mat_nby = np.meshgrid(list_nbx, list_nby)
    potentiel = np.round(np.sqrt(mat_nbx**2+mat_nby**2)-0.45)*ordre_v*1e300
    potentiely = 0
    video_simulation2D(parx, pary, potentiel, potentiely, allframe = 400,
                                fps=25, file_name='Simulation_2D_cercle_prop')